<a href="https://colab.research.google.com/github/Stella-Achar-Oiro/NLP-Sentinel-Analysis/blob/main/Stella_Oiro_NLP_Sentinel_Analysis_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#SETUP GPU
import tensorflow as tf
device_list = tf.test.gpu_device_name
print("Tensorflow version:", tf.__version__)

if tf.test.gpu_device_name():
    print("GPU is available")
else:
    print("GPU is NOT available")
device_list()

Tensorflow version: 2.12.0
GPU is available


'/device:GPU:0'

In [2]:
!pip install datasets transformers huggingface_hub
!apt-get install git-lfs

import pandas as pd
import numpy as np
#from datasets import load_metric
import os
import matplotlib.pyplot as plt
import wordcloud
from wordcloud import WordCloud, STOPWORDS, ImageColorGenerator
from sklearn.model_selection import train_test_split
from datasets import load_dataset, load_metric
import io
from transformers import AutoTokenizer, TrainingArguments, Trainer
tokenizer = AutoTokenizer.from_pretrained("bert-base-cased")
from google.colab import files
from google.colab import drive
from transformers import AutoModelForSequenceClassification
#converting training data to PyTorch tensors to speed up training and adding padding:
from transformers import DataCollatorWithPadding
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)
from huggingface_hub import notebook_login

# Disabe W&B
os.environ["WANDB_DISABLED"] = "true"

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 468.7/468.7 kB 12.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 65.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 200.1/200.1 kB 25.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 29.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.9/132.9 kB 17.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 7.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.2/212.2 kB 15.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 57.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 269.3/269.3 kB 26.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 158.8/158.8 kB 22.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.2/114.2 kB 15.4 MB/s eta 0:00:00
Reading package 

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [7]:
# Load the datasets

url = 'https://raw.githubusercontent.com/Azubi-Africa/Career_Accelerator_P5-NLP/master/zindi_challenge/data/SampleSubmission.csv'

df_sample = pd.read_csv(url)

url = 'https://raw.githubusercontent.com/Azubi-Africa/Career_Accelerator_P5-NLP/master/zindi_challenge/data/Test.csv'

df_test = pd.read_csv(url).fillna("")

url = 'https://raw.githubusercontent.com/Azubi-Africa/Career_Accelerator_P5-NLP/master/zindi_challenge/data/Train.csv'

df_train = pd.read_csv(url).dropna(axis = 0)

df_train.head()

df_test.head()

df_train.isnull().sum()

df_test.isnull().sum()

df_train, df_eval = train_test_split(df_train, test_size=0.2, random_state=42, stratify= df_train['label'])

print(f"New dataframe shapes: train is {df_train.shape}, eval is {df_eval.shape}")

New dataframe shapes: train is (7999, 4), eval is (2000, 4)


In [8]:
# Save splitted subsets
df_train.to_csv("/content/drive/MyDrive/P5/df_train_subset.csv", index=False)
df_eval.to_csv("/content/drive/MyDrive/P5/df_eval_subset.csv", index=False)

dataset = load_dataset('csv',
                        data_files={'df_train': '/content/drive/MyDrive/P5/df_train_subset.csv',
                        'df_eval': '/content/drive/MyDrive/P5/df_eval_subset.csv'}, encoding = "ISO-8859-1")

def transform_labels(label):

    label = label['label']
    num = 0
    if label == -1: #'Negative'
        num = 0
    elif label == 0: #'Neutral'
        num = 1
    elif label == 1: #'Positive'
        num = 2

    return {'labels': num}

def tokenize_data(example):
    return tokenizer(example['safe_text'], padding='max_length')

# Change the tweets to tokens that the models can exploit
dataset = dataset.map(tokenize_data, batched=True)

# Transform	labels and remove the useless columns
remove_columns = ['tweet_id', 'label', 'safe_text', 'agreement']
dataset = dataset.map(transform_labels, remove_columns=remove_columns)

dataset

training_args = TrainingArguments("test_trainer", num_train_epochs= 3, load_best_model_at_end=True,evaluation_strategy= "steps")

model = AutoModelForSequenceClassification.from_pretrained("bert-base-cased", num_labels=3)

train_dataset = dataset['df_train'].shuffle(seed=20) #.select(range(40000)) # to select a part
eval_dataset = dataset['df_eval'].shuffle(seed=20)

trainer = Trainer(
    model=model, args=training_args, train_dataset=train_dataset, eval_dataset=eval_dataset
)

trainer.train()

#defining the evaluation metrics
metric = load_metric("accuracy")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

trainer = Trainer(model=model, args=training_args, train_dataset=train_dataset, 
                  eval_dataset=eval_dataset,compute_metrics=compute_metrics)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

# Launch the final evaluation 
trainer.evaluate()


Extracting data files:   0%|          | 0/2 [00:00<?, ?it/s]

Generating df_train split: 0 examples [00:00, ? examples/s]

Generating df_eval split: 0 examples [00:00, ? examples/s]

Dataset csv downloaded and prepared to /root/.cache/huggingface/datasets/csv/default-c7f9b46fb85798f2/0.0.0/6954658bab30a358235fa864b05cf819af0e179325c740e4bc853bcc7ec513e1. Subsequent calls will reuse this data.


  0%|          | 0/2 [00:00<?, ?it/s]

...

Some weights of the model checkpoint at bert-base-cased were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at b

Step,Training Loss,Validation Loss
500,0.790800,0.686824
1000,0.698300,0.637631
1500,0.577100,0.634925
2000,0.545200,0.633084
2500,0.357300,0.813001
3000,0.367800,0.876342


<ipython-input-8-05ddc5d8497f>:48: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  metric = load_metric("accuracy")


You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


{'eval_loss': 0.6330835223197937,
 'eval_accuracy': 0.7505,
 'eval_runtime': 64.0141,
 'eval_samples_per_second': 31.243,
 'eval_steps_per_second': 3.905}

In [9]:
notebook_login()


Token is valid.
Your token has been saved in your configured git credential helpers (store).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [11]:
#saving model to hub
model.push_to_hub("Achar/NLP-Sentinel-Analysis")

#saving tokenizer to hub
tokenizer.push_to_hub("Achar/NLP-Sentinel-Analysis")

CommitInfo(commit_url='https://huggingface.co/Achar/NLP-Sentinel-Analysis/commit/92e18ff1b8c1746d5fd5b64d446b2dfc79f3a7f9', commit_message='Upload tokenizer', commit_description='', oid='92e18ff1b8c1746d5fd5b64d446b2dfc79f3a7f9', pr_url=None, pr_revision=None, pr_num=None)